# JuMP

JuMP is an _algebraic modeling language_ in Julia.

## Useful links

- Website: https://jump.dev
- Installation: https://www.juliaopt.org/JuMP.jl/stable/installation/
- Quickstart: https://www.juliaopt.org/JuMP.jl/stable/quickstart/
- Tutorials: https://github.com/JuliaOpt/JuMPTutorials.jl
- MathOptInterface. What? Why? How? http://www.optimization-online.org/DB_HTML/2020/02/7609.html

In [4]:
using JuMP
using Gurobi

model = Model(with_optimizer(Gurobi.Optimizer))
set_silent(model)

@variable(model, 0 <= x <= 2)
@variable(model, 0 <= y <= 30)

@objective(model, Max, 5x + 3y)

@constraint(model, my_con, x + 5y <= 3)

println(model)

Academic license - for non-commercial use only
Max 5 x + 3 y
Subject to
 x + 5 y ≤ 3.0
 x ≥ 0.0
 y ≥ 0.0
 x ≤ 2.0
 y ≤ 30.0



In [6]:
optimize!(model)

println()

@show JuMP.termination_status(model)
@show primal_status(model)
@show dual_status(model)
@show objective_value(model)
@show value(x)
@show value(y)
@show dual(my_con)
@show shadow_price(my_con)


JuMP.termination_status(model) = OPTIMAL::TerminationStatusCode = 1
primal_status(model) = FEASIBLE_POINT::ResultStatusCode = 1
dual_status(model) = FEASIBLE_POINT::ResultStatusCode = 1
objective_value(model) = 10.6
value(x) = 2.0
value(y) = 0.2
dual(my_con) = -0.6
shadow_price(my_con) = 0.6


0.6

JuMP's not just for linear programming. You can also create crazy models!

In [16]:
model = Model()
@variable(model, x[i = 1:4] >= i)
@variable(model, Y[1:2, 1:2], PSD)
@NLconstraint(model, sum(sin(x[i]) for i = 1:4) == π)
@objective(model, Min, [0.5, 0.5]' * Y * [0.5, 0.5] + sum(x))
model

A JuMP Model
Minimization problem with:
Variables: 7
Objective function type: GenericAffExpr{Float64,VariableRef}
`Array{VariableRef,1}`-in-`MathOptInterface.PositiveSemidefiniteConeTriangle`: 1 constraint
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 4 constraints
Nonlinear: 1 constraint
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.
Names registered in the model: Y, x

## Why should I use JuMP over...

- gurobipy? 

Not bound to one particular solver (but can still use Gurobi). Also supports nonlinear programming and conic problems (semidefinite programming, etc.)

- AMPL/GAMS/AIMMS/etc.?

Embedded in a high-level language. 

- PuLP?

Supports nonlinear, conic, callbacks, etc.

- Pyomo?

Interfaces with solvers via in-memory APIs. Can be orders of magnitude faster if you are solving problems sequentially.

- C/C++ APIs for CPLEX/Gurobi/...

Faster development time.

# SDDP.jl

## Hydro-thermal scheduling

## Problem Description

In a hydro-thermal problem, the agent controls a hydro-electric generator and reservoir.

Each time period, they need to choose a generation quantity from thermal `g_t`, and hydro
`g_h`, in order to meed demand `w_d`, which is a stagewise-independent random variable.

The state variable, `x`, is the quantity of water in the reservoir at the start of each
time period, and it has a minimum level of 5 units and a maximum level of 15 units. 

We
assume that there are 10 units of water in the reservoir at the start of time, so that
`x_0 = 10`. 

The state-variable is connected through time by the water balance constraint:
`x.out = x.in - g_h - s + w_i,` where `x.out` is the quantity of water at the end of the
time period, `x.in` is the quantity of water at the start of the time period, `s` is the
quantity of water spilled from the reservoir, and `w_i` is a stagewise-independent random
variable that represents the inflow into the reservoir during the time period.

We assume that there are three stages, `t=1, 2, 3`, representing summer-fall, winter, and
spring.

In each stage, the agent incurs the cost of spillage, plus the cost of thermal generation.

We assume that the cost of thermal generation is dependent on the stage `t = 1, 2, 3`, and
that in each stage, `w` is drawn from the set `(w_i, w_d) = {(0, 7.5), (3, 5), (10, 2.5)}`
with equal probability.

In [8]:
const x_lower = 5
const x_upper = 15
const x_0 = 10
const T = 3
const Ω = [(0, 7.5), (3, 5), (10, 2.5)]
const P = [1/3, 1/3, 1/3]

3-element Array{Float64,1}:
 0.3333333333333333
 0.3333333333333333
 0.3333333333333333

## Importing packages

For this example, in addition to `SDDP`, we need `Gurobi` as a solver, and `Statisitics` to
compute the mean of our simulations.

In [9]:
using Gurobi
using SDDP
using Statistics

# Initialize the Gurobi environment once.
const GRB_ENV = Gurobi.Env();

Academic license - for non-commercial use only


## Solving a finite horizon policy

Much of the macro code (i.e., lines starting with `@`) in the first part of the following
should be familiar to users of JuMP.

Inside the `do-end` block, `sp` is a standard JuMP model, and `t` is an
index for the state variable that will be called with `t = 1, 2, 3`.

The state variable `x`, constructed by passing the `SDDP.State` tag to `@variable` is
actually a Julia struct with two fields: `x.in` and `x.out` corresponding to the incoming
and outgoing state variables respectively. Both `x.in` and `x.out` are standard JuMP
variables. The `initial_value` keyword provides the value of the state variable in the
root node (i.e., `x_0`).

Compared to a JuMP model, one key difference is that we use `@stageobjective`
instead of `@objective`. The `SDDP.parameterize` function takes a list of supports
for `w` and parameterizes the JuMP model `sp` by setting the right-hand sides of the
appropriate constraints (note how the constraints initially have a right-hand side of
`0`). By default, it is assumed that the realizations have uniform probability, but a
probability mass vector can also be provided.

In [36]:
finite_horizon = SDDP.LinearPolicyGraph(
    stages = T, 
    sense = :Min, 
    lower_bound = 0.0, 
    optimizer = with_optimizer(Gurobi.Optimizer, GRB_ENV),
) do sp, t
    set_silent(sp)
    @variable(sp, x_lower <= x <= x_upper, SDDP.State, initial_value = x_0)
    @variable(sp, g_t >= 0)
    @variable(sp, g_h >= 0)
    @variable(sp, s >= 0)
    @constraint(sp, balance, x.out - x.in + g_h + s == 0)
    @constraint(sp, demand, g_h + g_t == 0)
    @stageobjective(sp, s + t * g_t)
    SDDP.parameterize(sp, Ω, P) do ω
        set_normalized_rhs(balance, ω[1])
        set_normalized_rhs(demand, ω[2])
    end
end

A policy graph with 3 nodes.
 Node indices: 1, 2, 3


## Training the policy

Once a model has been constructed, the next step is to train the policy. This can be
achieved using `SDDP.train`. There are many options that can be passed, but
`iteration_limit` terminates the training after the prescribed number of SDDP iterations.

In [37]:
SDDP.train(
    finite_horizon, 
    stopping_rules = [
        SDDP.Statistical(
            num_replications = 100, 
            iteration_period = 20, 
            z_score = 1.96,
            verbose = true
        )
    ]
)

--------------------------------------------------------------------------------
                      SDDP.jl (c) Oscar Dowson, 2017-20

Numerical stability report
  Non-zero Matrix range     [1e+00, 1e+00]
  Non-zero Objective range  [1e+00, 3e+00]
  Non-zero Bounds range     [5e+00, 2e+01]
  Non-zero RHS range        [3e+00, 1e+01]
No problems detected

Solver: serial mode

 Iteration    Simulation       Bound         Time (s)    Proc. ID   # Solves
        1    1.500000e+01   9.166667e+00   2.403021e-03          1         12
        2    2.500000e+00   1.037037e+01   3.660917e-03          1         24
        3    4.000000e+01   1.051440e+01   5.056858e-03          1         36
        4    1.351587e+01   1.076543e+01   9.335995e-03          1         48
        5    7.333333e+00   1.092593e+01   1.128888e-02          1         60
        6    2.500000e+00   1.098148e+01   1.454282e-02          1         72
        7    2.500000e+00   1.098148e+01   1.648402e-02          1         

## Simulating the policy

After training, we can simulate the policy using `SDDP.simulate`.

In [38]:
sims = SDDP.simulate(finite_horizon, 1_000, [:x, :g_t])
mu = round(mean([s[1][:g_t] for s in sims]), digits = 2)
println("On average, $(mu) units of thermal are used in the first stage.")

On average, 1.58 units of thermal are used in the first stage.


`sims` is a vector of vectors of dictionaries. It has the following content:

In [39]:
replication = 1
stage = 2
sims[replication][stage]

Dict{Symbol,Any} with 8 entries:
  :g_t             => 0.0
  :bellman_term    => 8.5
  :noise_term      => (3, 5)
  :node_index      => 2
  :stage_objective => 0.0
  :objective_state => nothing
  :belief          => Dict(2=>1.0)
  :x               => State{Float64}(7.5, 5.5)

We can visualize the solution using a spaghetti plot (you'll see why it has this name).

In [40]:
p = SDDP.SpaghettiPlot(sims)
SDDP.add_spaghetti(p, title = "Reservoir Level") do data 
    return data[:x].out
end
SDDP.add_spaghetti(p, title = "Thermal Generation") do data 
    return data[:g_t]
end
SDDP.add_spaghetti(p, title = "Inflow") do data 
    return data[:noise_term][1]
end
SDDP.add_spaghetti(p, title = "Demand") do data 
    return data[:noise_term][2]
end
SDDP.plot(p)

## Extracting the water values

Finally, we can use `SDDP.ValueFunction` and `SDDP.evaluate` to obtain and
evaluate the value function at different points in the state-space. Note that since we
are minimizing, the price has a negative sign: each additional unit of water leads to a
decrease in the the expected long-run cost.

In [41]:
V = SDDP.ValueFunction(finite_horizon[1])
set_silent(V.model)
cost, price = SDDP.evaluate(V, x = 0.5 * (x_lower + x_upper))

(8.055555555555555, Dict(:x=>-0.555556))

In [42]:
SDDP.plot(V, x = x_lower:0.05:x_upper)

## Solving an infinite horizon policy

There are three stages in our problem, so we construct a linear policy graph with three
stages using `SDDP.LinearGraph`:

In [43]:
graph = SDDP.LinearGraph(T)

Root
 0
Nodes
 1
 2
 3
Arcs
 0 => 1 w.p. 1.0
 1 => 2 w.p. 1.0
 2 => 3 w.p. 1.0


Then, because we want to solve an infinite-horizon problem, we add an additional edge
between node `3` and node `1` with probability `0.95`:

In [44]:
SDDP.add_edge(graph, T => 1, 0.95)
graph

Root
 0
Nodes
 1
 2
 3
Arcs
 0 => 1 w.p. 1.0
 1 => 2 w.p. 1.0
 2 => 3 w.p. 1.0
 3 => 1 w.p. 0.95


In [45]:
infinite_horizon = SDDP.PolicyGraph(
    graph, 
    sense = :Min, 
    lower_bound = 0.0, 
    optimizer = with_optimizer(Gurobi.Optimizer, GRB_ENV),
) do sp, t
    set_silent(sp)
    @variable(sp, x_lower <= x <= x_upper, SDDP.State, initial_value = x_0)
    @variables(sp, begin
        g_t >= 0
        g_h >= 0
        s   >= 0
        inflow  # This time we add inflow as a slack variable
        demand  # Same for demand.
    end)
    @constraints(sp, begin
        x.out == x.in - g_h - s + inflow
        g_h + g_t == demand
    end)
    @stageobjective(sp, s + t * g_t)
    SDDP.parameterize(sp, Ω, P) do ω
        fix(inflow, ω[1])
        fix(demand, ω[2])
    end
end
SDDP.train(infinite_horizon, iteration_limit = 100)
V = SDDP.ValueFunction(infinite_horizon[1])
set_silent(V.model)
cost, price = SDDP.evaluate(V, x = 10)

--------------------------------------------------------------------------------
                      SDDP.jl (c) Oscar Dowson, 2017-20

Numerical stability report
  Non-zero Matrix range     [1e+00, 1e+00]
  Non-zero Objective range  [1e+00, 3e+00]
  Non-zero Bounds range     [5e+00, 2e+01]
  Non-zero RHS range        [0e+00, 0e+00]
No problems detected

Solver: serial mode

 Iteration    Simulation       Bound         Time (s)    Proc. ID   # Solves
        1    9.150000e+01   3.889139e+01   1.446295e-02          1        111
        2    4.306198e+02   1.068403e+02   5.641985e-02          1        510
        3    1.750000e+01   1.108152e+02   5.791306e-02          1        525
        4    1.374177e+02   1.214596e+02   6.619287e-02          1        612
        5    2.735418e+02   1.513640e+02   8.719206e-02          1        879
        6    1.402386e+02   1.592942e+02   9.405208e-02          1        966
        7    2.950000e+02   1.795309e+02   1.144390e-01          1       12

(233.5343971228139, Dict(:x=>-0.658945))

In [46]:
SDDP.plot(V, x = x_lower:0.05:x_upper)